In [1]:
import tensorflow as tf
from skimage.io import imread
import matplotlib.pyplot as plt
import numpy as np
import Utils
import os
import keras
%matplotlib inline

Using TensorFlow backend.


In [2]:
vgg16 = keras.applications.VGG16(include_top=False,weights='imagenet')
vgg16_graph = keras.backend.get_session().graph

In [3]:
w_1_1 = vgg16_graph.get_tensor_by_name('block1_conv1/kernel:0')
b_1_1 = vgg16_graph.get_tensor_by_name('block1_conv1/bias:0')
w_1_2 = vgg16_graph.get_tensor_by_name('block1_conv2/kernel:0')
b_1_2 = vgg16_graph.get_tensor_by_name('block1_conv2/bias:0')

w_2_1 = vgg16_graph.get_tensor_by_name('block2_conv1/kernel:0')
b_2_1 = vgg16_graph.get_tensor_by_name('block2_conv1/bias:0')
w_2_2 = vgg16_graph.get_tensor_by_name('block2_conv2/kernel:0')
b_2_2 = vgg16_graph.get_tensor_by_name('block2_conv2/bias:0')

w_3_1 = vgg16_graph.get_tensor_by_name('block3_conv1/kernel:0')
b_3_1 = vgg16_graph.get_tensor_by_name('block3_conv1/bias:0')
w_3_2 = vgg16_graph.get_tensor_by_name('block3_conv2/kernel:0')
b_3_2 = vgg16_graph.get_tensor_by_name('block3_conv2/bias:0')
w_3_3 = vgg16_graph.get_tensor_by_name('block3_conv3/kernel:0')
b_3_3 = vgg16_graph.get_tensor_by_name('block3_conv3/bias:0')

w_4_1 = vgg16_graph.get_tensor_by_name('block4_conv1/kernel:0')
b_4_1 = vgg16_graph.get_tensor_by_name('block4_conv1/bias:0')
w_4_2 = vgg16_graph.get_tensor_by_name('block4_conv2/kernel:0')
b_4_2 = vgg16_graph.get_tensor_by_name('block4_conv2/bias:0')
w_4_3 = vgg16_graph.get_tensor_by_name('block4_conv3/kernel:0')
b_4_3 = vgg16_graph.get_tensor_by_name('block4_conv3/bias:0')

w_5_1 = vgg16_graph.get_tensor_by_name('block5_conv1/kernel:0')
b_5_1 = vgg16_graph.get_tensor_by_name('block5_conv1/bias:0')
w_5_2 = vgg16_graph.get_tensor_by_name('block5_conv2/kernel:0')
b_5_2 = vgg16_graph.get_tensor_by_name('block5_conv2/bias:0')
w_5_3 = vgg16_graph.get_tensor_by_name('block5_conv3/kernel:0')
b_5_3 = vgg16_graph.get_tensor_by_name('block5_conv3/bias:0')

In [4]:
number_of_classes = 12
batch_size = 8
input_shape = (batch_size, 352, 480, 3)
label_shape = (batch_size, 352, 480, number_of_classes)

images=tf.placeholder(dtype=tf.float32,shape=input_shape)
labels_im=tf.placeholder(dtype=tf.float32,shape=label_shape)

#images=tf.placeholder(dtype=tf.float32,shape=(1, 720, 960, 3))
strides = (1,1,1,1)
padding = 'SAME'
max_pooling_ksize = (1,2,2,1)
learning_rate = 0.0001
num_epochs = 150

In [5]:
def unpool_with_argmax(pool, ind, name = None, ksize=[1, 2, 2, 1]):

    """
       Unpooling layer after max_pool_with_argmax.
       Args:
           pool:   max pooled output tensor
           ind:      argmax indices
           ksize:     ksize is the same as for the pool
       Return:
           unpool:    unpooling tensor
    """
    with tf.variable_scope(name):
        input_shape = pool.get_shape().as_list()
        print("input_shape: {0}, ind: {1}".format(pool.shape, ind.shape))
        output_shape = (input_shape[0], input_shape[1] * ksize[1], input_shape[2] * ksize[2], input_shape[3])

        flat_input_size = tf.reduce_prod(input_shape)
        flat_output_shape = [output_shape[0], output_shape[1] * output_shape[2] * output_shape[3]]

        pool_ = tf.reshape(pool, [flat_input_size])
        batch_range = tf.reshape(tf.range(output_shape[0], dtype=ind.dtype), shape=[input_shape[0], 1, 1, 1])
        b = tf.ones_like(ind) * batch_range
        b = tf.reshape(b, [flat_input_size, 1])
        ind_ = tf.reshape(ind, [flat_input_size, 1])
        ind_ = tf.concat([b, ind_], 1)

        ret = tf.scatter_nd(ind_, pool_, shape=flat_output_shape)
        ret = tf.reshape(ret, output_shape)
        return ret

In [6]:
def encoding_layer(input, filter1, filter2, bias1, bias2, conv_stride, name=None, filter3=None, bias3=None, max_pool_stride = (1,2,2,1)):
    with tf.name_scope(name):
        conv_1= tf.layers.batch_normalization(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input = input,filter=filter1, strides=conv_stride, padding=padding), bias1)))
        conv_2= tf.layers.batch_normalization(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input = conv_1,filter=filter2, strides=conv_stride, padding=padding), bias2)))
        if filter3 != None and bias3 != None:
            conv_3 = tf.layers.batch_normalization(tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input = conv_2,filter=filter3, strides=conv_stride, padding=padding), bias3)))
            return tf.nn.max_pool_with_argmax(conv_3, ksize=max_pooling_ksize,strides=(1,2,2,1), padding='VALID')
        else:
            return tf.nn.max_pool_with_argmax(conv_2, ksize=max_pooling_ksize,strides=(1,2,2,1), padding='VALID')

def decoding_layer(input, indicies, filter1_shape, filter2_shape, name=None, filter3_shape=None):
    with tf.name_scope(name):
        upsample = unpool_with_argmax(input, indicies, name=name+"Upsample")
        if filter3_shape != None:
            deconv_1 = tf.layers.batch_normalization(tf.layers.conv2d(upsample, filter3_shape,(3,3),activation=tf.nn.relu, padding='same'))
            deconv_2 = tf.layers.batch_normalization(tf.layers.conv2d(deconv_1, filter2_shape,(3,3),activation=tf.nn.relu, padding='same'))
            return tf.layers.batch_normalization(tf.layers.conv2d(deconv_2, filter1_shape,(3,3),activation=tf.nn.relu, padding='same'))
        else:
            deconv_1 = tf.layers.batch_normalization(tf.layers.conv2d(upsample, filter2_shape,(3,3),activation=tf.nn.relu, padding='same'))
            return tf.layers.batch_normalization(tf.layers.conv2d(deconv_1, filter1_shape,(3,3),activation=tf.nn.relu, padding='same'))
        
        

In [7]:
max_pooling_1, indicies_1 = encoding_layer(images, w_1_1, w_1_2, b_1_1, b_1_2, conv_stride=strides, name="Encoding_layer_1")

max_pooling_2, indicies_2 = encoding_layer(max_pooling_1, w_2_1, w_2_2, b_2_1, b_2_2, conv_stride=strides, name="Encoding_layer_2")
max_pooling_3, indicies_3 = encoding_layer(max_pooling_2, w_3_1, w_3_2, b_3_1, b_3_2, conv_stride=strides, name="Encoding_layer_3", filter3=w_3_3, bias3=b_3_3)
max_pooling_4, indicies_4 = encoding_layer(max_pooling_3, w_4_1, w_4_2, b_4_1, b_4_2, conv_stride=strides, name="Encoding_layer_4", filter3=w_4_3, bias3=b_4_3)
max_pooling_5, indicies_5 = encoding_layer(max_pooling_4, w_5_1, w_5_2, b_5_1, b_5_2, conv_stride=strides, name="Encoding_layer_5", filter3=w_5_3, bias3=b_5_3)

deconv_1 = decoding_layer(max_pooling_5, indicies_5, w_5_1.shape[2].value, w_5_2.shape[2].value, name= "Decoding_layer_1", filter3_shape=w_5_3.shape[2].value)
deconv_2 = decoding_layer(deconv_1, indicies_4, w_4_1.shape[2].value, w_4_2.shape[2].value, name= "Decoding_layer_2", filter3_shape=w_4_3.shape[2].value)
deconv_3 = decoding_layer(deconv_2, indicies_3, w_3_1.shape[2].value, w_3_2.shape[2].value, name= "Decoding_layer_3", filter3_shape=w_3_3.shape[2].value)
deconv_4 = decoding_layer(deconv_3, indicies_2, w_2_1.shape[2].value, w_2_2.shape[2].value, name= "Decoding_layer_4")
deconv_5 = decoding_layer(deconv_4, indicies_1, number_of_classes, w_1_2.shape[2].value, name= "Decoding_layer_5")

out = tf.nn.softmax(deconv_5)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=labels_im))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

correctPred = tf.equal(tf.argmax(out,-1), tf.argmax(labels_im,-1))
print("correctPred: {0}".format(correctPred.shape))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

train_accu = tf.summary.scalar("train_accu", accuracy)

train_loss = tf.summary.scalar("train_loss", loss)

train_summ = tf.summary.merge((train_accu, train_loss))


input_shape: (2, 11, 15, 512), ind: (2, 11, 15, 512)
input_shape: (2, 22, 30, 512), ind: (2, 22, 30, 512)
input_shape: (2, 44, 60, 256), ind: (2, 44, 60, 256)
input_shape: (2, 88, 120, 128), ind: (2, 88, 120, 128)
input_shape: (2, 176, 240, 64), ind: (2, 176, 240, 64)
correctPred: (2, 352, 480)


In [8]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
writer = tf.summary.FileWriter("tensorboard_logs")
writer.add_graph(sess.graph)
saver = tf.train.Saver()

In [9]:
train_images = np.asarray(Utils.load_images("../Data/CamVid/train/"))
test_images = np.asarray(Utils.load_images("../Data/CamVid/test/"))

In [10]:
train_annotations = np.asarray(Utils.load_annotations("../Data/CamVid/trainannot/"))
test_annotations = np.asarray(Utils.load_annotations("../Data/CamVid/testannot/"))

In [11]:
training_bp = Utils.BatchProcessor()
test_bp = Utils.BatchProcessor()

In [12]:
for i in range(int(num_epochs*len(train_images) / batch_size)):
    batch_X, batch_y = training_bp.get_next_batch(train_images, train_annotations, batch_size)
    
    sess.run(train_step,feed_dict={images: batch_X,labels_im: batch_y})
    
    if i % 50 == 0:
        accus = []
        losss = []
        for j in range(int(len(test_images)/batch_size)):
            test_batch_x, test_batch_y = test_bp.get_next_batch(test_images, test_annotations, batch_size)
            loss_o, accu_o = sess.run([loss,accuracy],feed_dict={images: test_batch_x,labels_im: test_batch_y})
            accus.append(accu_o)
            losss.append(loss_o)
        train_summary = sess.run(train_summ,feed_dict={images: batch_X,labels_im: batch_y})
        writer.add_summary(train_summary, i)
        
        test_sum = tf.Summary()
        test_sum.value.add(tag="TestAccuracy", simple_value =np.mean(accus))
        writer.add_summary(test_sum, i)
        print("{2}/{3} Test accu: {0}, test loss: {1}".format(np.mean(accus), np.mean(losss), i, int(num_epochs*len(train_images) / batch_size)))
    
    if i % 1000 == 0 and i!=0:
        save_path = saver.save(sess, os.path.join('..','saved_models','model.ckpt'))
        print("Model saved to: %s" % save_path)


0/18350 Test accu: 0.25138533115386963, test loss: 2.4848742485046387
50/18350 Test accu: 0.2549406886100769, test loss: 2.363787889480591


KeyboardInterrupt: 